## Exercise 6: Simon's Algorithm


_course: quantum cryptography for beginners
<br>date: 14 october 2020
<br>author: burton rosenberg_

###  Simon's Algorithm

This project builds on the last project that explored the Bernstein-Vazirani algorithm.

It is a bit of an odd problem to be interested. It is really about exploring techniques available in quantum computing. And hence here we go, exploring.

We will be again making use of the excluse or operation. We write $a \oplus b$ for the bitwise exclusive-or of base 2 representations of the numbers $a$ and $b$. We are considering a function $f$ that is an exact 2-to-1 map,

$$
f(a) = f(b) \mbox{ if and only if } a\oplus b = r.
$$

This necessarily means that the size of the domain is exactly twice that of the range. And example of such a function is and $f$ that drops the least significant bit of its input,

$$
f(a) = \lfloor a/2 \rfloor
$$

In this case, $f(a) = f(b)$ when $a\oplus b =1 $, i.e. differs only in the least significant digit. For inputs from 0 to $2^n-1$ the output is from 0 to $2^{n-1}-1$.

Simon's problem is, given a black-box implementation of $f$, find $r$. That is, we are given a device that will evaluate $f$ on a presented output. After a number of evaluations we discover $r$.


### Classical computation requires exponential time


Classically, we have to evaluate on any squence of inputs, $x_1, x_2, \ldots$ until by good fortune we find $f(x_i)=f(x_j)$. Actually we can consider this problem from the perspective of the black-box. As $x_i$ are presented, unless for some previous $x_j$ the equation $x_i = r \oplus x_j$, the black-box is free to pick any output not yet used and assign it to $f(x_i)$. Hence we have a problem in first collision in hashing on the presentaton of $x_i$.

So when about $\sqrt{N}$ values been presented, the probability of a collision is $\sqrt{N}/N= 1/\sqrt{N}$. Hence after an additional $\sqrt{N}$ values are presented, the expected number of collisions is 1. 

### The idea

The function $f$ is implemented as a quantum gate $U_r$ on $2n-1$ qubits, with the first $n$ qubits being $x$ and the last $n-1$ qubits receiving an ex-or of $f(x)$. This is the general format of having ancilla qubits record results reversibly by using an ex-or.

The gate $U_r$ then receives a superposition of all values on the first $n$ qubits and zero on the last $n-1$ qubits. The last $n-1$ qubits are measured. Now the state of the unmeasured qubits must be consistent with the measurement. That is, from,

$$
U_f\bigl(\sum|0\rangle |i\rangle\bigr) = \sum U_f\bigl(|i\rangle\bigr)|i\rangle
$$

the state collapses after measurement to,

$$
|y\rangle |a\rangle + |y\rangle |b\rangle
$$

for the unique $a$ and $b$ such that $f(a)=f(b)=y$.

We now have another case of quantum coyness. The answer is there, but quantum refuses to let us know. If we measure now, on this superposed state, we can learn either $a$ or $b$, but on learning one, we must learn nothing about the other. So we have no further information on $r a\oplus b$. We need to pass this superposed state through a bank of Hadamards to cause interference between the bit representations of $a$ and $b$, which gives a clue as to $r$.

The mathematics here gets a bit more involved. Remember that for $a$, the $H^{\otimes n}(a)$ will be a superposition of all $|i\rangle$ with a particular pattern of plus and minus weights. Same is true of $H^{\otimes n}(b)$. That they are in input superposition then results in cancelations for any $|i\rangle$ where the signs in the representation differ. Hence on measuring, we are sure not to measure any $|i\rangle$ for which this cancelation occurs. 

We we can show is that what we can get is any $|j\rangle$ such that $[j|r]=0$.

_Notation:_ $[i|j]$ is a computation where $i$ and $j$ are considered bit vectors of equal length, and we calculate a dot product over $F_2^n$. To see what this means, I have implemented the function as xor-dot-product.

Example: $[6|3]$ would write $6=110$ and $3=011$, and the see that they have on location with both bits 1. Since there is an odd number of locations where both have bit one, the result is 1, $[6|3]=1$.


### A bit of the complicated math

The dot product we are using is a linear function in the addition $\oplus$, so,

$$
[(a \oplus r)| j ]= [a| j] \oplus [r | j ]
$$

We write out the superposition,

\begin{eqnarray}
H^{\otimes n}(a+b) &=& \sum_j (-1)^{[a| j]}|j\rangle  +  
\sum_j (-1)^{[b| j]}|j\rangle  \\
&=& \sum_j \bigl( (-1)^{[a|j]} + (-1)^{[b| j]}\bigr) |j\rangle \\
\end{eqnarray}

we understand that the only $|j\rangle$ what will be measured are those with non-zero probability to be measured,

\begin{eqnarray}
(-1)^{[a| j]} + (-1)^{[b| j]} &=&
   (-1)^{[a | j]} + (-1)^{[(a\oplus r) | j]} \\
&=&  (-1)^{[a | j]}\bigl(1+(-1)^{[r | j]}\bigr) \\
&\ne& 0
\end{eqnarray}

which means we will measure those $|j\rangle$ for which  $[r | j]=0$.

### So where do we go from here

By running the experiment $k$ times, we sample find qubit values $j_1, j_2, \ldots , j_k$  that determine $r$ through the hints,


$$
\{\, j_i\, |\, [j_i | r] = 0, \;i=1,\dots,k\,\}
$$


We will simply look for an $r$ that satisfies these $k$ constraints. 

However, we could more efficiently find such an $r$ using linear algebra. Also, from linear algebra we also know about how many experiements we need to run. We skip over this as it is a separate topic really, with the only thing to know at this moment, is such an $r$ an be found in time $O(k^3)$.

Hence Simons problem that would take time $\Theta(2^{k/2})$ classically can be solved on a quantum computer with about $k$ quantum experiments followed by an $O(k^3)$ classical computation.


### Creating the functions. 

To experiment with Simon's algorithm, we will need functions for which $f(a)=f(b)$ exactly when $a\oplus b = r$ for some non-zero $r$. Recall that $\oplus$ is the bitwise exclusive or. If also $a\oplus b'=r$, we have

\begin{eqnarray}
a\oplus b &=& a\oplus b',\\
a\oplus( a\oplus b) &=& a\oplus (a \oplus b') \;\mbox{ adding $a$ to both sides},\\ 
(a\oplus a)\oplus b &=& (a\oplus )a \oplus b'\\
0\oplus b &=& 0\oplus b'\\
b &=& b'
\end{eqnarray}

Hence there can be only two values which fit. If the input is $n$ bits, or qubits, the $2^n$ values pair into $2^{n-1}$ pairs, and each pair maps to a unique output.

For $n=2$, we can create three functions as the dot product between the input at each of the three numbers $01, 10$ and $11$. 

The following code demonstrates this. There are a few things to notice about the code. 

The first is the so-called xor-inner product code. That should explain how to calculate these inner products. The code works bit by bit on integers i and j. If the product of i and j is odd, then both are odd, if it is even then one or the other or both were even. Masking off this bit with &amp;1 we cumulatively x-or it into s. Then shift down i and j by integer divide by 2 to get the next bit. This continues until one or the other of what remains of i or j is zero.

The second is the use of a lambda expression. The func_to_map takes a function on one variable as an argument, and we supply the function by first using a def to create a function on possibly many variables, and then us a lamba to turn that into a function on one variable, as required by func_to_map.




In [1]:

def xor_dot_product(x,y):
    """
    computes [x|y] defined as the dot product computed in F_2^n
    """
    s = 0
    while x*y != 0:
        s ^= (x*y)&1
        x, y = x//2, y//2
    return s
   
def func_to_map_n(func):
    """
    given a function func = (f,n_bits) on integers of size 0 to 2**n_bits-1, return a 
    map presentation of the function 
       f_as_map[i]  = [{all j such that f(j)==i}]
    """
    f, n_bits = func
    f_as_map = {}
    for i in range(2**n_bits):
        j = f(i)
        if j not in f_as_map:
            f_as_map[j] = []
        f_as_map[j].append(i)
    return f_as_map

def print_map_n(m):
    for t in m:
        print(f"\t{t}: {m[t]}")
    print()
       
def xor_bit_function(u):
    """
    wrap the xor-dot with u into a function on 2 bits
    returns a function on one variable
    """
    return ((lambda n: xor_dot_product(u,n),2))
 
    
possible_simon_map = [
    func_to_map_n(xor_bit_function(1)),
    func_to_map_n(xor_bit_function(2)),
    func_to_map_n(xor_bit_function(3))
]

description_of_map = [
    "group by low order bit (even/odd)",
    "group by high order bit (lower half/upper half)",
    "group by parity (even/odd number of one's)"
]

for s,m in zip(description_of_map,possible_simon_map):
    print(s)
    print_map_n(m)


group by low order bit (even/odd)
	0: [0, 2]
	1: [1, 3]

group by high order bit (lower half/upper half)
	0: [0, 1]
	1: [2, 3]

group by parity (even/odd number of one's)
	0: [0, 3]
	1: [1, 2]



We can now create a curcuit for the first part of Simon's algorithm by implementing the exor-inner-product with CNOT's, placing the output on an ancilla bit, as we must in order that the computation be reversable.

Understand why the results confim the above maps. We have yet to make this output usable, however. Recall that we do not want to repeatedly measure until we find two elements of a pair. While in this case, it is easy to find two elements of a pair, because there are so few pairs, with $2^k$ values of outputs, it would take $2^{k/2}$ tries to find two elements of a pair, and this is exponential time.

In [2]:
import qiskit
import time, math

from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.jobstatus import JOB_FINAL_STATES, JobStatus
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

qiskit.__qiskit_version__

args_g = []

# your api token from IBM, first time run.
# after that None is good

#api_token = 'abcdefghijklmnopqrstuvwxyz'
api_token = None 

def load_or_save_IBMQ_account(api_token=None):
    global args_g
    if api_token: 
    # only needs to be done once
        # then is stored in e.g. ~/.qistkit/qiskitrc
        IBMQ.save_account(api_token)
    provider = IBMQ.load_account()
# choose an alternative provider, if available
#    provider = IBMQ.providers()[1]
    return provider

def list_backends(provider):
    global args_g
    backends = provider.backends()
    print('backends available:')
    for be in backends:
        st = be.status()
        if st.operational:
            print(f'\t{be.name()}, pending jobs:{st.pending_jobs}')

            
def run_quantum_circuit_on_backend(quantum_circuit,provider,backend):
    backend = provider.get_backend(backend)
    qobj = assemble(transpile(quantum_circuit, backend=backend), backend=backend)
    job = backend.run(qobj)
    return job


def wait_for_job(backend, job, wait_interval=5):
    backend = provider.get_backend(backend)
    retrieved_job = backend.retrieve_job(job.job_id())
    start_time = time.time()
    job_status = job.status()
    while job_status not in JOB_FINAL_STATES:
        print(f'Status @ {time.time() - start_time:0.0f} s: {job_status.name},'
              f' est. queue position: {job.queue_position()}')
        time.sleep(wait_interval)
        job_status = job.status()


print("listing backends ...")
provider = load_or_save_IBMQ_account(api_token)
list_backends(provider)

# choose your backend

backend = 'ibmq_qasm_simulator'
#backend = 'ibmq_armonk'
#backend = 'ibmq_vigo'
#backend = 'ibmq_london'
#backend = 'ibmq_ourense'

# and so forth ... chose from the results given by provider.backends()

listing backends ...


C:\Users\boltj\Anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


backends available:
	ibmq_qasm_simulator, pending jobs:2
	ibmqx2, pending jobs:71
	ibmq_16_melbourne, pending jobs:66
	ibmq_vigo, pending jobs:230
	ibmq_ourense, pending jobs:39
	ibmq_valencia, pending jobs:162
	ibmq_armonk, pending jobs:15
	ibmq_athens, pending jobs:24
	ibmq_santiago, pending jobs:53


In [68]:
#backend = 'ibmq_santiago'

In [69]:
qreg_q = QuantumRegister(3, 'q')
creg_c = ClassicalRegister(3, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.h(qreg_q[0])
circuit.h(qreg_q[1])
circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2])
circuit.cx(qreg_q[0], qreg_q[2])
circuit.barrier(qreg_q[1], qreg_q[0], qreg_q[2])
circuit.measure(qreg_q[0], creg_c[0])
circuit.measure(qreg_q[1], creg_c[1])
circuit.measure(qreg_q[2], creg_c[2])


print('\n-------- CIRCUIT ---------')
print(circuit.draw(output='text'))
print('-------------------------\n')


job = run_quantum_circuit_on_backend(circuit,provider,backend)
print(f'results: waiting for results from backend {backend} ...')
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')

circuit = QuantumCircuit(qreg_q, creg_c)
circuit.h(qreg_q[0])
circuit.h(qreg_q[1])
circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2])
circuit.cx(qreg_q[1], qreg_q[2])
circuit.barrier(qreg_q[1], qreg_q[0], qreg_q[2])
circuit.measure(qreg_q[0], creg_c[0])
circuit.measure(qreg_q[1], creg_c[1])
circuit.measure(qreg_q[2], creg_c[2])


print('\n-------- CIRCUIT ---------')
print(circuit.draw(output='text'))
print('-------------------------\n')


job = run_quantum_circuit_on_backend(circuit,provider,backend)
print(f'results: waiting for results from backend {backend} ...')
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')


circuit = QuantumCircuit(qreg_q, creg_c)
circuit.h(qreg_q[0])
circuit.h(qreg_q[1])
circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2])
circuit.cx(qreg_q[0], qreg_q[2])
circuit.cx(qreg_q[1], qreg_q[2])
circuit.barrier(qreg_q[1], qreg_q[0], qreg_q[2])
circuit.measure(qreg_q[0], creg_c[0])
circuit.measure(qreg_q[1], creg_c[1])
circuit.measure(qreg_q[2], creg_c[2])


print('\n-------- CIRCUIT ---------')
print(circuit.draw(output='text'))
print('-------------------------\n')


job = run_quantum_circuit_on_backend(circuit,provider,backend)
print(f'results: waiting for results from backend {backend} ...')
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')




-------- CIRCUIT ---------
     ┌───┐ ░       ░ ┌─┐      
q_0: ┤ H ├─░───■───░─┤M├──────
     ├───┤ ░   │   ░ └╥┘┌─┐   
q_1: ┤ H ├─░───┼───░──╫─┤M├───
     └───┘ ░ ┌─┴─┐ ░  ║ └╥┘┌─┐
q_2: ──────░─┤ X ├─░──╫──╫─┤M├
           ░ └───┘ ░  ║  ║ └╥┘
c: 3/═════════════════╩══╩══╩═
                      0  1  2 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'000': 251, '010': 245, '101': 281, '111': 247}

-------- CIRCUIT ---------
     ┌───┐ ░       ░ ┌─┐      
q_0: ┤ H ├─░───────░─┤M├──────
     ├───┤ ░       ░ └╥┘┌─┐   
q_1: ┤ H ├─░───■───░──╫─┤M├───
     └───┘ ░ ┌─┴─┐ ░  ║ └╥┘┌─┐
q_2: ──────░─┤ X ├─░──╫──╫─┤M├
           ░ └───┘ ░  ║  ║ └╥┘
c: 3/═════════════════╩══╩══╩═
                      0  1  2 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'000': 259, '001': 246, '110

### Exercise A: 

Here is a family of functions from 3-bit to 2-bit integers, 

$$
f_{u,v}(n) = [n|u] + 2*[n|v]
$$

where $u$ and $v$ are 3-bit integers. 

This functions are implementable by using CNOT's from three input qubits to two ancilla qubits.

I have implemented this function in the following  code, and show, for every $u,v$ pair, what the inverse function looks like. The inverse function is held as a map,

$$
i\rightarrow \{\, j\, |\, f_{u,v}(j) = i \,\}
$$

1. The function inverse groups into 1 group of 8, 2 groups of 4, or 4 groups of 2. Describe what at the conditions dermining the grouping.
2. For the 4 groups of 2 determine that there is an period $r$ such that for each pair $(a,b)$ then $a\oplus b = r$.
3. Make a table that for the seven non-zero values of $r$ provides the $(u,v)$ values that create a function $f_{u,v}$ whose pairs have period $r$.
4. Select and make a vector of an $(u,v)$ for each $r$.

__Note on programming__: The lambda expression is an anonymous function. The define xor-pair-function bakes in u and v values, and a combining formula, and returns a function ready to evaluate when given a single input. The input will bind to the n in the lambda, and the body of the lamba will evaluate with that n, and the u and v that were available when the lambda was created. 

Note that the function is pair, with the second element the number of bits in the input. Some of the code needs to know now this number.  


In [3]:

def xor_pair_function(u,v):
    """
    given u,v in the range 0-7, returns a function f:[0-7]->[0-3]
    where f(n) = 2*bit_1 + bit_0 and
        bit 0: [u|n]
        bit 1: [v|n]
    
    returned object is a pair (f,3)
    """
    return (lambda n: xor_dot_product(u,n)+2*xor_dot_product(v,n),3)

def listall_xor_pair_function():
    for u in range(8):
        for v in range(8):
            func = xor_pair_function(u,v)
            m = func_to_map_n(func)
            print(f'function is lsb [n|{u}], msb [n|{v}]')
            print_map_n(m)


listall_xor_pair_function()



def get_r_from_function(func):
    m = func_to_map_n(func)
    if(len(m.keys()) == 4):
        r = m[0][0] ^ m[0][1]
    else:
        r = 0
    return r

def functions_by_r():
    by_r = {}
    for u in range(8):
        for v in range(8):
            func = xor_pair_function(u,v)
            m = func_to_map_n(func)
            r = get_r_from_function(func)
            by_r[r] = m
    return by_r

def print_functions_by_r(by_r):
    for r in sorted(by_r.keys()):
        print (f'{r}: {by_r[r]}')

print_functions_by_r(functions_by_r())


# you. might want to decide on functions for each of the r values,
# and place them in a list

simon_functions = {1:xor_pair_function(2,4),
                  2:xor_pair_function(1,4),
                  3:xor_pair_function(3,4),
                  4:xor_pair_function(1,2),
                  5:xor_pair_function(2,5),
                  6:xor_pair_function(1,6),
                  7:xor_pair_function(3,5),}


function is lsb [n|0], msb [n|0]
	0: [0, 1, 2, 3, 4, 5, 6, 7]

function is lsb [n|0], msb [n|1]
	0: [0, 2, 4, 6]
	2: [1, 3, 5, 7]

function is lsb [n|0], msb [n|2]
	0: [0, 1, 4, 5]
	2: [2, 3, 6, 7]

function is lsb [n|0], msb [n|3]
	0: [0, 3, 4, 7]
	2: [1, 2, 5, 6]

function is lsb [n|0], msb [n|4]
	0: [0, 1, 2, 3]
	2: [4, 5, 6, 7]

function is lsb [n|0], msb [n|5]
	0: [0, 2, 5, 7]
	2: [1, 3, 4, 6]

function is lsb [n|0], msb [n|6]
	0: [0, 1, 6, 7]
	2: [2, 3, 4, 5]

function is lsb [n|0], msb [n|7]
	0: [0, 3, 5, 6]
	2: [1, 2, 4, 7]

function is lsb [n|1], msb [n|0]
	0: [0, 2, 4, 6]
	1: [1, 3, 5, 7]

function is lsb [n|1], msb [n|1]
	0: [0, 2, 4, 6]
	3: [1, 3, 5, 7]

function is lsb [n|1], msb [n|2]
	0: [0, 4]
	1: [1, 5]
	2: [2, 6]
	3: [3, 7]

function is lsb [n|1], msb [n|3]
	0: [0, 4]
	3: [1, 5]
	2: [2, 6]
	1: [3, 7]

function is lsb [n|1], msb [n|4]
	0: [0, 2]
	1: [1, 3]
	2: [4, 6]
	3: [5, 7]

function is lsb [n|1], msb [n|5]
	0: [0, 2]
	3: [1, 3]
	2: [4, 6]
	1: [5, 7]

function is l

### Exercise B

The quantum computer part of exercise A.


1. Select for each $r$ on of the functions $f_(u,v)$ and make a quantum circuit realizing it.
2. Run the circuit, and write a program to analyze the resulting samples to reproduce the pairs calculated by the inverse map.



__Note on 5:__ If you use the simulator, the results will be easier to interpret. As an additional exercise below, you will be asked to write code that understands the errors made by they actual quantum devices.

The output we get from the simulator or device is a count array with a five character string in the characters 0 and 1. These are the measurements, associated right to left with the qubit diagram top to bottom. I.e., the least significant binary bit is the measurement of the topmost qubit or classical bit.

Opps, maybe I should be careful. I always measure with the i-th classical bit being the i-th qubit; and I have never had to think which of the two indexes the output. Probably the classical.

If you have constructed your gates as I have, the leftmost two bits are a binary representation of the funnction output and the rightmost three bits are the binary representation of the function input. Converting them to integers, construct a dictionary, keyed on function output with value a list of inputs giving that output.



In [4]:

def simon_first_half(fuv):
    qreg_q = QuantumRegister(5, 'q')
    creg_c = ClassicalRegister(5, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.h(qreg_q[0])
    circuit.h(qreg_q[1])
    circuit.h(qreg_q[2])
    circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])

    #circuit.cx(qreg_q[0], qreg_q[4])
    fuv(circuit,qreg_q)
    
    circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])
    for i in range(5):
        circuit.measure(qreg_q[i], creg_c[i])

    return circuit



def fuv_1(c,q):
    c.cx(q[1], q[3])
    c.cx(q[2], q[4])
    return c

def fuv_2(c,q):
    c.cx(q[0], q[3])
    c.cx(q[2], q[4])
    return c

def fuv_3(c,q):
    c.cx(q[0], q[3])
    c.cx(q[1], q[3])
    c.cx(q[2], q[4])
    return c
    
def fuv_4(c,q):
    c.cx(q[0], q[3])
    c.cx(q[1], q[4])
    return c
    
def fuv_5(c,q):
    c.cx(q[1], q[4])
    c.cx(q[0], q[3])
    c.cx(q[1], q[3])
    c.cx(q[2], q[4])
    return c

def fuv_6(c,q):
    c.cx(q[0], q[3])
    c.cx(q[1], q[3])
    c.cx(q[1], q[4])
    c.cx(q[2], q[4])
    return c

def fuv_7(c,q):
    c.cx(q[1], q[3])
    c.cx(q[0], q[3])
    c.cx(q[1], q[4])
    c.cx(q[2], q[4])
    return c

fuv = [ fuv_1, fuv_2, fuv_3, fuv_4, fuv_5, fuv_6, fuv_7]
fuv_desc = ['xor 1','xor 2','xor 3','xor 4','xor 5','xor 6','xor 7']

def organize_counts(counts):
    """
    this function interprets the output of the quantum runs
    """
    d = {"00": 0,
  "01": 0,
  "10": 0,
        "11": 0}
    for x in counts:
        if x[0] == "0" and  x[1] == "0":
            d["00"] = d["00"] + counts[x]
        elif x[0] == "0" and  x[1] == "1":
            d["01"] = d["01"] + counts[x]
        elif x[0] == "1" and  x[1] == "0":
            d["10"] = d["10"] + counts[x]
        elif x[0] == "1" and  x[1] == "1":
            d["11"] = d["11"] + counts[x]
    
    return d


def run_circuits(circuits):
    results = []
    for circuit in circuits:
        print('\n-------- CIRCUIT ---------')
        print(circuit.draw(output='text'))
        print('-------------------------\n')
        job = run_quantum_circuit_on_backend(circuit,provider,backend)
        print(f'results: waiting for results from backend {backend} ...')
        wait_for_job(backend, job)
        result = job.result()
        results.append(result.get_counts())
        print(f'results: {results[-1]}')
    return results
        

simon_first_half_results = run_circuits([simon_first_half(uf) for uf in fuv])
for i in range(len(simon_first_half_results)):
    print(f'{fuv_desc[i]}: {organize_counts(simon_first_half_results[i])}')


-------- CIRCUIT ---------
     ┌───┐ ░            ░ ┌─┐            
q_0: ┤ H ├─░────────────░─┤M├────────────
     ├───┤ ░            ░ └╥┘┌─┐         
q_1: ┤ H ├─░───■────────░──╫─┤M├─────────
     ├───┤ ░   │        ░  ║ └╥┘┌─┐      
q_2: ┤ H ├─░───┼────■───░──╫──╫─┤M├──────
     └───┘ ░ ┌─┴─┐  │   ░  ║  ║ └╥┘┌─┐   
q_3: ──────░─┤ X ├──┼───░──╫──╫──╫─┤M├───
           ░ └───┘┌─┴─┐ ░  ║  ║  ║ └╥┘┌─┐
q_4: ──────░──────┤ X ├─░──╫──╫──╫──╫─┤M├
           ░      └───┘ ░  ║  ║  ║  ║ └╥┘
c: 5/══════════════════════╩══╩══╩══╩══╩═
                           0  1  2  3  4 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'00000': 143, '00001': 130, '10100': 129, '10101': 131, '11110': 124, '11111': 127, '01010': 103, '01011': 137}

-------- CIRCUIT ---------
     ┌───┐ ░            ░ ┌─┐            
q_0: ┤ H ├─░───■────────░─┤M├────────────
     ├───┤ ░   │        ░ └╥┘┌─┐         
q_1: 


### Exercise C

In order to extract the information about $r$, the Hadamard transformation is applied. The transformation for basis state $i$ is,

$$
H^{\otimes n}|i\rangle = \sum_j (-1)^{[i|j]}\,|j\rangle
$$

Finish the program that produces the list where the $j$ element on the list is the $j$ coeeficient, $+1$ or $-1$, in this transformation.

Then add and list those $|j\rangle$ that can be measured (non-zero probability) for the various pairs for each $r$. 

Ascertain that the each such $j$ has $[r|j]=0$.


In [5]:

def hadamard_coefficients(i,n_bits):
    hc = []
    for x in range(2**n_bits):
        if(xor_dot_product(x,i) != 0):
            hc.append(-1)
        else:
            hc.append(1)  
    return hc


def hadamard_superposed(i1,i2,n_bits):
    return [(hadamard_coefficients(i1,n_bits)[j] 
            + hadamard_coefficients(i2,n_bits)[j])//2 for j in range(2**n_bits) ]
  
def possible_observations(func):
    m = func_to_map_n(func)
    possible_obs = [m]
    
    
    
    return possible_obs

def verify_observations_against_r(obs,r):
    ans = True
    for o in obs:
        if xor_dot_product(o,r)!=0:
            print(f'observation {o} is not null against r={r}')
            ans = False
    return ans
    
def test_hadamard_coefficients(n_bits):
    soln_3 = {0: [1, 1, 1, 1, 1, 1, 1, 1],
        1: [1, -1, 1, -1, 1, -1, 1, -1],
        2: [1, 1, -1, -1, 1, 1, -1, -1],
        3: [1, -1, -1, 1, 1, -1, -1, 1],
        4: [1, 1, 1, 1, -1, -1, -1, -1],
        5: [1, -1, 1, -1, -1, 1, -1, 1],
        6: [1, 1, -1, -1, -1, -1, 1, 1],
        7: [1, -1, -1, 1, -1, 1, 1, -1]}
    for i in range(2**n_bits):
        assert(hadamard_coefficients(i,3)==soln_3[i])
        print(f"{i}: {hadamard_coefficients(i,3)}")
    return True


test_hadamard_coefficients(3)

def listall_possible_observations(the_functions):
    print(f'\npossible observations,\n\tby period of function')
    for r in the_functions:
        obs = possible_observations(the_functions[r])
        print(f'\t{r}: {obs}')
        
listall_possible_observations(simon_functions)
        


0: [1, 1, 1, 1, 1, 1, 1, 1]
1: [1, -1, 1, -1, 1, -1, 1, -1]
2: [1, 1, -1, -1, 1, 1, -1, -1]
3: [1, -1, -1, 1, 1, -1, -1, 1]
4: [1, 1, 1, 1, -1, -1, -1, -1]
5: [1, -1, 1, -1, -1, 1, -1, 1]
6: [1, 1, -1, -1, -1, -1, 1, 1]
7: [1, -1, -1, 1, -1, 1, 1, -1]

possible observations,
	by period of function
	1: [{0: [0, 1], 1: [2, 3], 2: [4, 5], 3: [6, 7]}]
	2: [{0: [0, 2], 1: [1, 3], 2: [4, 6], 3: [5, 7]}]
	3: [{0: [0, 3], 1: [1, 2], 2: [4, 7], 3: [5, 6]}]
	4: [{0: [0, 4], 1: [1, 5], 2: [2, 6], 3: [3, 7]}]
	5: [{0: [0, 5], 2: [1, 4], 1: [2, 7], 3: [3, 6]}]
	6: [{0: [0, 6], 1: [1, 7], 2: [2, 4], 3: [3, 5]}]
	7: [{0: [0, 7], 3: [1, 6], 1: [2, 5], 2: [3, 4]}]


### Exercise D

The quantum computing part of exercise C.

The full Simon algorithm circuit is provided, without the circuitry that builds the 2-to-1 function. That you have already provided as a list of functions in the previous exercises.

Get the output and show that it is correct. You can use the simulator to avoide spurious answers. A later exercise will ask about running the algorithm on actual hardware, and adjusting to the errors that might occur.




In [6]:

def simon_full_circuit(fuv):
    qreg_q = QuantumRegister(5, 'q')
    creg_c = ClassicalRegister(5, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.h(qreg_q[0])
    circuit.h(qreg_q[1])
    circuit.h(qreg_q[2])
    circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])

    #circuit.cx(qreg_q[0], qreg_q[4])
    fuv(circuit,qreg_q)
    
    circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])
    circuit.h(qreg_q[0])
    circuit.h(qreg_q[1])
    circuit.h(qreg_q[2])
    circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])
  
    for i in range(5):
        circuit.measure(qreg_q[i], creg_c[i])

    return circuit

simon_full_results = run_circuits([simon_full_circuit(uf) for uf in fuv])

def analyze_simon_full(counts):
    d = {"00": 0,
  "01": 0,
  "10": 0,
        "11": 0}
    for x in counts:
        if x[0] == "0" and  x[1] == "0":
            d["00"] = d["00"] + counts[x]
        elif x[0] == "0" and  x[1] == "1":
            d["01"] = d["01"] + counts[x]
        elif x[0] == "1" and  x[1] == "0":
            d["10"] = d["10"] + counts[x]
        elif x[0] == "1" and  x[1] == "1":
            d["11"] = d["11"] + counts[x]
    
    return d


for i in range(len(simon_full_results)):
    print(f'{fuv_desc[i]}: {analyze_simon_full(simon_full_results[i])}')


-------- CIRCUIT ---------
     ┌───┐ ░            ░ ┌───┐ ░ ┌─┐            
q_0: ┤ H ├─░────────────░─┤ H ├─░─┤M├────────────
     ├───┤ ░            ░ ├───┤ ░ └╥┘┌─┐         
q_1: ┤ H ├─░───■────────░─┤ H ├─░──╫─┤M├─────────
     ├───┤ ░   │        ░ ├───┤ ░  ║ └╥┘┌─┐      
q_2: ┤ H ├─░───┼────■───░─┤ H ├─░──╫──╫─┤M├──────
     └───┘ ░ ┌─┴─┐  │   ░ └───┘ ░  ║  ║ └╥┘┌─┐   
q_3: ──────░─┤ X ├──┼───░───────░──╫──╫──╫─┤M├───
           ░ └───┘┌─┴─┐ ░       ░  ║  ║  ║ └╥┘┌─┐
q_4: ──────░──────┤ X ├─░───────░──╫──╫──╫──╫─┤M├
           ░      └───┘ ░       ░  ║  ║  ║  ║ └╥┘
c: 5/══════════════════════════════╩══╩══╩══╩══╩═
                                   0  1  2  3  4 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'00000': 63, '10000': 58, '10010': 65, '10100': 58, '10110': 71, '11000': 52, '11010': 59, '11100': 59, '11110': 75, '00010': 58, '00100': 69, '00110': 66, '01000': 71



### Exercise E

The result of some $O(n)$ applications of Simons algorithm is $n$ observations $o_i$ such that $[o_i|r]=$. From this we need to calculate $r$.

The easiest but non-efficient method is to check for each $r$ if it does have zero dot product with each $o_i$. Please implement that first, and check that we have concluded a (non-efficient) version of Simon's algorithm.

How better to find $r$? If there is no better way than what we has been suggested in the previous paragraphs, we have not done better than classical computing. However this is the problem of finding a solution to simultaneous linear equations, and can be done in time $O(n^2)$. Using this result, we have an exponential speed up over the classical algorithm.

I will discuss the linear algebra the does this, and work an example. 


In [7]:
def all_zero_dot_product(basis,n_bits):
    
    def all_zero(b):
        for e in b:
            if e!=0:
                return False
        return True
    for i in range(2**n_bits):
        r = xor_dot_product(basis,i)
    return r

all_zero_dot_product(2,3)

1

### Extra problems 


### Exercise F

The data from a quantum computer is noisey, for these circuits. Adapt your summarizing functions to deal with bad outputs.

### Exercise G

Is it possible to extend this approach to larger input sizes. Is it possible to have a 4 bit to 3 bit function of the type required by using three usages of the ex-or functions?



In [8]:
backend = 'ibmq_santiago'

def simon_full_circuit(fuv):
    qreg_q = QuantumRegister(5, 'q')
    creg_c = ClassicalRegister(5, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.h(qreg_q[0])
    circuit.h(qreg_q[1])
    circuit.h(qreg_q[2])
    circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])

    #circuit.cx(qreg_q[0], qreg_q[4])
    fuv(circuit,qreg_q)
    
    circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])
    circuit.h(qreg_q[0])
    circuit.h(qreg_q[1])
    circuit.h(qreg_q[2])
    circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2], qreg_q[3], qreg_q[4])
  
    for i in range(5):
        circuit.measure(qreg_q[i], creg_c[i])

    return circuit

simon_full_results = run_circuits([simon_full_circuit(uf) for uf in fuv])

def analyze_simon_full(counts):
    d = {"00": 0,
  "01": 0,
  "10": 0,
        "11": 0}
    for x in counts:
        if x[0] == "0" and  x[1] == "0":
            d["00"] = d["00"] + counts[x]
        elif x[0] == "0" and  x[1] == "1":
            d["01"] = d["01"] + counts[x]
        elif x[0] == "1" and  x[1] == "0":
            d["10"] = d["10"] + counts[x]
        elif x[0] == "1" and  x[1] == "1":
            d["11"] = d["11"] + counts[x]
    
    return d


for i in range(len(simon_full_results)):
    print(f'{fuv_desc[i]}: {analyze_simon_full(simon_full_results[i])}')


-------- CIRCUIT ---------
     ┌───┐ ░            ░ ┌───┐ ░ ┌─┐            
q_0: ┤ H ├─░────────────░─┤ H ├─░─┤M├────────────
     ├───┤ ░            ░ ├───┤ ░ └╥┘┌─┐         
q_1: ┤ H ├─░───■────────░─┤ H ├─░──╫─┤M├─────────
     ├───┤ ░   │        ░ ├───┤ ░  ║ └╥┘┌─┐      
q_2: ┤ H ├─░───┼────■───░─┤ H ├─░──╫──╫─┤M├──────
     └───┘ ░ ┌─┴─┐  │   ░ └───┘ ░  ║  ║ └╥┘┌─┐   
q_3: ──────░─┤ X ├──┼───░───────░──╫──╫──╫─┤M├───
           ░ └───┘┌─┴─┐ ░       ░  ║  ║  ║ └╥┘┌─┐
q_4: ──────░──────┤ X ├─░───────░──╫──╫──╫──╫─┤M├
           ░      └───┘ ░       ░  ║  ║  ║  ║ └╥┘
c: 5/══════════════════════════════╩══╩══╩══╩══╩═
                                   0  1  2  3  4 
-------------------------

results: waiting for results from backend ibmq_santiago ...
Status @ 0 s: VALIDATING, est. queue position: None
Status @ 5 s: QUEUED, est. queue position: 13
Status @ 11 s: QUEUED, est. queue position: 12
Status @ 16 s: QUEUED, est. queue position: 12
Status @ 22 s: QUEUED, est. queue position:

Status @ 280 s: QUEUED, est. queue position: 3
Status @ 285 s: QUEUED, est. queue position: 3
Status @ 290 s: QUEUED, est. queue position: 3
Status @ 296 s: QUEUED, est. queue position: 2
Status @ 301 s: QUEUED, est. queue position: 2
Status @ 306 s: QUEUED, est. queue position: 2
Status @ 311 s: QUEUED, est. queue position: 1
Status @ 317 s: QUEUED, est. queue position: 1
Status @ 322 s: RUNNING, est. queue position: None
Status @ 327 s: RUNNING, est. queue position: None
Status @ 333 s: RUNNING, est. queue position: None
Status @ 338 s: RUNNING, est. queue position: None
results: {'00000': 72, '00001': 71, '10000': 67, '10001': 58, '10010': 3, '10011': 3, '10100': 63, '10101': 32, '10110': 4, '10111': 4, '11000': 72, '11001': 60, '11010': 4, '11011': 1, '11100': 49, '11101': 41, '11110': 4, '11111': 3, '00010': 3, '00011': 10, '00100': 62, '00101': 57, '00110': 5, '00111': 4, '01000': 75, '01001': 78, '01010': 10, '01011': 3, '01100': 60, '01101': 40, '01110': 2, '01111': 4}

-------

Status @ 259 s: QUEUED, est. queue position: 5
Status @ 264 s: QUEUED, est. queue position: 5
Status @ 270 s: QUEUED, est. queue position: 5
Status @ 275 s: QUEUED, est. queue position: 5
Status @ 280 s: QUEUED, est. queue position: 5
Status @ 286 s: QUEUED, est. queue position: 5
Status @ 291 s: QUEUED, est. queue position: 5
Status @ 296 s: QUEUED, est. queue position: 5
Status @ 302 s: QUEUED, est. queue position: 5
Status @ 307 s: QUEUED, est. queue position: 5
Status @ 312 s: QUEUED, est. queue position: 5
Status @ 317 s: QUEUED, est. queue position: 5
Status @ 322 s: QUEUED, est. queue position: 5
Status @ 328 s: QUEUED, est. queue position: 5
Status @ 333 s: QUEUED, est. queue position: 5
Status @ 338 s: QUEUED, est. queue position: 5
Status @ 343 s: QUEUED, est. queue position: 5
Status @ 349 s: QUEUED, est. queue position: 5
Status @ 354 s: QUEUED, est. queue position: 5
Status @ 359 s: QUEUED, est. queue position: 5
Status @ 364 s: QUEUED, est. queue position: 5
Status @ 370 

results: waiting for results from backend ibmq_santiago ...
Status @ 0 s: VALIDATING, est. queue position: None
Status @ 5 s: QUEUED, est. queue position: 10
Status @ 11 s: QUEUED, est. queue position: 10
Status @ 16 s: QUEUED, est. queue position: 10
Status @ 21 s: QUEUED, est. queue position: 10
Status @ 27 s: QUEUED, est. queue position: 10
Status @ 32 s: QUEUED, est. queue position: 9
Status @ 37 s: QUEUED, est. queue position: 9
Status @ 42 s: QUEUED, est. queue position: 9
Status @ 48 s: QUEUED, est. queue position: 8
Status @ 53 s: QUEUED, est. queue position: 8
Status @ 58 s: QUEUED, est. queue position: 8
Status @ 63 s: QUEUED, est. queue position: 8
Status @ 69 s: QUEUED, est. queue position: 8
Status @ 74 s: QUEUED, est. queue position: 8
Status @ 79 s: QUEUED, est. queue position: 8
Status @ 85 s: QUEUED, est. queue position: 8
Status @ 90 s: QUEUED, est. queue position: 8
Status @ 95 s: QUEUED, est. queue position: 8
Status @ 100 s: QUEUED, est. queue position: 7
Status @ 

In [ ]:
Messed with counts a little

xor 1: {'00': 306, '01': 266, '10': 272, '11': 180}
xor 2: {'00': 284, '01': 272, '10': 234, '11': 234}
xor 3: {'00': 257, '01': 251, '10': 269, '11': 247}
xor 4: {'00': 319, '01': 325, '10': 190, '11': 190}
xor 5: {'00': 257, '01': 238, '10': 248, '11': 281}
xor 6: {'00': 284, '01': 259, '10': 239, '11': 242}
xor 7: {'00': 247, '01': 284, '10': 252, '11': 241}